In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.9 MB/s eta 0:00:00


In [2]:
!pip install lmoments3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import scipy.stats as st
from scipy import stats
from scipy.stats import shapiro
from scipy.stats import anderson
from scipy.stats import kstest
from scipy.stats import jarque_bera
from statsmodels.stats.diagnostic import lilliefors
from scipy.stats import normaltest  #D`Agostino
from scipy.stats import chisquare
import seaborn as sns
from sklearn import tree
from scipy.stats import differential_entropy, norm
import statistics
import scipy.special
from catboost import CatBoostClassifier
import lmoments3 as lm
import math

In [4]:
def featureHesapla(vec):
  feature=[]
  feature.append(np.mean(vec))
  feature.append(st.kurtosis(vec))
  feature.append(st.skew(vec))
  feature.append(np.quantile(vec, 0.125))
  feature.append(np.quantile(vec, 0.25))
  feature.append(np.quantile(vec, 0.375))
  feature.append(np.quantile(vec, 0.625))
  feature.append(np.quantile(vec, 0.75))
  feature.append(np.quantile(vec,0.875))
  feature.append(differential_entropy(vec))
  feature.append(len(vec))
  feature.append(np.min(vec))
  feature.append(np.max(vec))

  l_1,l_2,l_skewness,l_kurtosis=lm.lmom_ratios(vec,nmom=4)
  feature.append(l_1)
  feature.append(l_2)
  feature.append(l_skewness)
  feature.append(l_kurtosis)

  return feature

In [5]:
def setUret2(dagilimId,orneklemBuyuklugu,tekrar):
  myDataSet=[]
  myLabels=[]
  for k in range(len(dagilimId)):
    for j in range(len(orneklemBuyuklugu)):
      for i in range(tekrar[k][j]):
        if dagilimId[k]==0: myData = list(np.random.normal(size=orneklemBuyuklugu[j]))
        if dagilimId[k]==1: myData = list(np.random.beta(2,2, size=orneklemBuyuklugu[j]))
        if dagilimId[k]==2: myData = list(np.random.logistic(0,1,size=orneklemBuyuklugu[j]))
        if dagilimId[k]==3: myData = list(np.random.standard_t(5,size=orneklemBuyuklugu[j]))
        if dagilimId[k]==4: myData = list(np.random.uniform(0,1,size=orneklemBuyuklugu[j]))
        if dagilimId[k]==5: myData = list(np.random.laplace(size=orneklemBuyuklugu[j]))
        if dagilimId[k]==6: myData =list(np.random.beta(2,5, size=orneklemBuyuklugu[j]))
        if dagilimId[k]==7: myData = list(np.random.gamma(4,5, size=orneklemBuyuklugu[j]))
        if dagilimId[k]==8: myData =list(np.random.lognormal(size=orneklemBuyuklugu[j]))
        if dagilimId[k]==9: myData = list(np.random.chisquare(5, size=orneklemBuyuklugu[j]))
        if dagilimId[k]==10: myData = list(np.random.exponential(size=orneklemBuyuklugu[j]))

        myDataStd=(myData-np.mean(myData))/np.std(myData)
        myDataSort=np.sort(myDataStd)
        myFeatures = featureHesapla(myDataSort)
        myDataSet.append(myFeatures)
        if dagilimId[k]==0:
          myLabels.append(1)
        else:
          myLabels.append(0)
  return myDataSet,myLabels

In [6]:
def setUret(dagilimId,orneklemBuyuklugu,tekrar):
  myDataSet=[]
  myLabels=[]
  ll_Results=[]
  ad_Results=[]
  dap_Results=[]
  jb_Results=[]
  sw_Results=[]

  for k in range(len(dagilimId)):
    for j in orneklemBuyuklugu:
      for i in range(tekrar[k]):
        if dagilimId[k]==0: myData = list(np.random.normal(size=j))
        if dagilimId[k]==1: myData = list(np.random.beta(2,2, size=j))
        if dagilimId[k]==2: myData = list(np.random.logistic(0,1,size=j))
        if dagilimId[k]==3: myData = list(np.random.standard_t(5,size=j))
        if dagilimId[k]==4: myData = list(np.random.uniform(0,1,size=j))
        if dagilimId[k]==5: myData = list(np.random.laplace(size=j))
        if dagilimId[k]==6: myData =list(np.random.beta(2,5, size=j))
        if dagilimId[k]==7: myData = list(np.random.gamma(4,5, size=j))
        if dagilimId[k]==8: myData =list(np.random.lognormal(size=j))
        if dagilimId[k]==9: myData = list(np.random.chisquare(5, size=j))
        if dagilimId[k]==10: myData = list(np.random.exponential(size=j))

        myDataStd=(myData-np.mean(myData))/np.std(myData)
        myDataSort=np.sort(myDataStd)
        myFeatures = featureHesapla(myDataSort)
        myDataSet.append(myFeatures)
        if dagilimId[k]==0:
          myLabels.append(1)
        else:
          myLabels.append(0)

        stat_ll, ll_p_value = lilliefors(myDataSort)
        if ll_p_value < 0.05:
          ll_Results.append(1)
        else:
          ll_Results.append(0)

        result = anderson(myDataSort,dist="norm")
        if result[1][2]<result[0]:
          ad_Results.append(1)
        else:
          ad_Results.append(0)

        stat_dap, dap_p_value = normaltest(myDataSort)
        if dap_p_value < 0.05:
          dap_Results.append(1)
        else:
          dap_Results.append(0)

        stat_jb, jb_p_value = jarque_bera(myDataSort)
        if jb_p_value < 0.05:
          jb_Results.append(1)
        else:
          jb_Results.append(0)

        stat_sw, sw_p_value = shapiro(myDataSort)
        if sw_p_value < 0.05:
          sw_Results.append(1)
        else:
          sw_Results.append(0)

  return myDataSet,myLabels,ll_Results, ad_Results, dap_Results, jb_Results, sw_Results

In [7]:
#orijinel
from sklearn.utils import class_weight
orneklemBuyuklugu=[10,20,30,40,50,100,500]
tekrar=[[3500,2800,2800,1600,1600,1000,100],
        [10,10,10,10,10,20,30],
        [700,550,350,250,200,100,100],
        [600,550,550,250,200,200,100],
        [10,10,10,20,30,30,10],
        [200,30,30,30,30,30,30],
        [10,10,10,10,10,30,30],
        [10,10,10,10,10,20,10],
        [20,20,20,10,10,10,10],
        [10,10,10,20,20,20,10],
        [10,20,20,20,20,10,10]]

dagilimId=[0,1,2,3,4,5,6,7,8,9,10]

X, y = setUret2(dagilimId=dagilimId,
               orneklemBuyuklugu=orneklemBuyuklugu,
               tekrar=tekrar)

xN= pd.DataFrame(X)

rfc = CatBoostClassifier(iterations=100)
rfc_model = rfc.fit(X,y)

Learning rate set to 0.300541
0:	learn: 0.6211488	total: 56.4ms	remaining: 5.59s
1:	learn: 0.5785004	total: 63.5ms	remaining: 3.11s
2:	learn: 0.5545937	total: 70.5ms	remaining: 2.28s
3:	learn: 0.5402358	total: 77.2ms	remaining: 1.85s
4:	learn: 0.5300190	total: 83.8ms	remaining: 1.59s
5:	learn: 0.5229467	total: 90.5ms	remaining: 1.42s
6:	learn: 0.5190900	total: 97ms	remaining: 1.29s
7:	learn: 0.5150124	total: 104ms	remaining: 1.19s
8:	learn: 0.5126489	total: 111ms	remaining: 1.12s
9:	learn: 0.5109301	total: 117ms	remaining: 1.06s
10:	learn: 0.5083885	total: 124ms	remaining: 1s
11:	learn: 0.5062744	total: 131ms	remaining: 963ms
12:	learn: 0.5046004	total: 138ms	remaining: 922ms
13:	learn: 0.5033363	total: 145ms	remaining: 889ms
14:	learn: 0.5029082	total: 152ms	remaining: 859ms
15:	learn: 0.5013403	total: 158ms	remaining: 831ms
16:	learn: 0.5006148	total: 166ms	remaining: 809ms
17:	learn: 0.4998698	total: 173ms	remaining: 787ms
18:	learn: 0.4989204	total: 180ms	remaining: 766ms
19:	learn

In [ ]:
from sklearn.utils import class_weight
orneklemBuyuklugu=[10,20,30,40,50,100,500]
tekrar=[[1500,1100,800,800,800,400,100],
        [50,50,50,50,50,50,50],
        [50,50,50,50,50,50,50],
        [50,50,50,50,50,50,50],
        [50,50,50,50,50,50,50],
        [50,50,50,50,50,50,50],
        [50,50,50,50,50,50,50],
        [50,50,50,50,50,50,50],
        [50,50,50,50,50,50,50],
        [50,50,50,50,50,50,50],
        [50,50,50,50,50,50,50]]

dagilimId=[0,1,2,3,4,5,6,7,8,9,10]

X, y = setUret2(dagilimId=dagilimId,
               orneklemBuyuklugu=orneklemBuyuklugu,
               tekrar=tekrar)

xN= pd.DataFrame(X)

rfc = CatBoostClassifier(iterations=100)
rfc_model = rfc.fit(X,y)

Learning rate set to 0.217468
0:	learn: 0.6012614	total: 9.82ms	remaining: 972ms
1:	learn: 0.5369499	total: 20.7ms	remaining: 1.01s
2:	learn: 0.4920076	total: 25.7ms	remaining: 832ms
3:	learn: 0.4590276	total: 30.8ms	remaining: 738ms
4:	learn: 0.4354515	total: 35.8ms	remaining: 681ms
5:	learn: 0.4146650	total: 40.8ms	remaining: 639ms
6:	learn: 0.4008758	total: 46.2ms	remaining: 613ms
7:	learn: 0.3902976	total: 51.2ms	remaining: 589ms
8:	learn: 0.3824609	total: 56.3ms	remaining: 569ms
9:	learn: 0.3744787	total: 61.5ms	remaining: 553ms
10:	learn: 0.3681743	total: 66.6ms	remaining: 539ms
11:	learn: 0.3640333	total: 71.6ms	remaining: 525ms
12:	learn: 0.3591008	total: 76.5ms	remaining: 512ms
13:	learn: 0.3556464	total: 81.7ms	remaining: 502ms
14:	learn: 0.3526832	total: 87.1ms	remaining: 494ms
15:	learn: 0.3488391	total: 92.1ms	remaining: 484ms
16:	learn: 0.3467633	total: 97.4ms	remaining: 475ms
17:	learn: 0.3446370	total: 102ms	remaining: 467ms
18:	learn: 0.3426452	total: 108ms	remaining: 

In [8]:
testorneklemBuyuklugu=[10,20,30,40,50,100,500]
veriSayisi=[10]
testdagilimId=[0]
testTekrar=100


for n in testorneklemBuyuklugu:
  myPerformances=[]
  ll_Performances=[]
  ad_Performances=[]
  dap_Performances=[]
  jb_Performances=[]
  sw_Performances=[]
  for m in range(testTekrar):
    testX, testY, ll, ad, dap, jb, sw = setUret(dagilimId=testdagilimId,orneklemBuyuklugu=[n],tekrar=veriSayisi)
    xN= pd.DataFrame(testX)
    predictedY = rfc.predict(xN)
    acc= sum(predictedY==testY)/len(predictedY)
    myPerformances.append(acc)

    acc_ll=sum(ll)/len(ll)
    acc_ad=sum(ad)/len(ad)
    acc_dap=sum(dap)/len(dap)
    acc_jb=sum(jb)/len(jb)
    acc_sw=sum(sw)/len(sw)

    ll_Performances.append(acc_ll)
    ad_Performances.append(acc_ad)
    dap_Performances.append(acc_dap)
    jb_Performances.append(acc_jb)
    sw_Performances.append(acc_sw)

  if testdagilimId[0]==0:
    hata=np.round(1-np.mean(myPerformances),5)
    print(n ,' orneklem buyuklugu icin MLPN Tip I hata = ' ,hata )
    print(n, ' orneklem buyuklugu icin LL testinin Tip I hatası = ' , np.round(np.mean(ll_Performances),5))
    print(n,' orneklem buyuklugu icin AD testinin Tip I hatası = ' , np.round(np.mean(ad_Performances),5))
    print(n,' orneklem buyuklugu icin DAP testinin Tip I hatası = ' , np.round(np.mean(dap_Performances),5))
    print(n,' orneklem buyuklugu icin JB testinin Tip I hatası = ' , np.round(np.mean(jb_Performances),5))
    print(n,' orneklem buyuklugu icin SW testinin Tip I hatası = ' , np.round(np.mean(sw_Performances),5),'\n')
  else:
    print(n ,' orneklem buyuklugu icin MLPN güç = ' , np.round(np.mean(myPerformances),5))
    print(n, ' orneklem buyuklugu icin LL testinin gücü = ' , np.round(np.mean(ll_Performances),5))
    print(n,' orneklem buyuklugu icin AD testinin gücü = ' , np.round(np.mean(ad_Performances),5))
    print(n,' orneklem buyuklugu icin DAP testinin gücü = ' , np.round(np.mean(dap_Performances),5))
    print(n,' orneklem buyuklugu icin JB testinin gücü = ' , np.round(np.mean(jb_Performances),5))
    print(n,' orneklem buyuklugu icin SW testinin gücü = ' , np.round(np.mean(sw_Performances),5),'\n')


/usr/local/lib/python3.10/dist-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  res = hypotest_fun_out(*samples, **kwds)


10  orneklem buyuklugu icin MLPN Tip I hata =  0.05
10  orneklem buyuklugu icin LL testinin Tip I hatası =  0.044
10  orneklem buyuklugu icin AD testinin Tip I hatası =  0.042
10  orneklem buyuklugu icin DAP testinin Tip I hatası =  0.051
10  orneklem buyuklugu icin JB testinin Tip I hatası =  0.007
10  orneklem buyuklugu icin SW testinin Tip I hatası =  0.048 

20  orneklem buyuklugu icin MLPN Tip I hata =  0.056
20  orneklem buyuklugu icin LL testinin Tip I hatası =  0.06
20  orneklem buyuklugu icin AD testinin Tip I hatası =  0.062
20  orneklem buyuklugu icin DAP testinin Tip I hatası =  0.061
20  orneklem buyuklugu icin JB testinin Tip I hatası =  0.021
20  orneklem buyuklugu icin SW testinin Tip I hatası =  0.049 

30  orneklem buyuklugu icin MLPN Tip I hata =  0.041
30  orneklem buyuklugu icin LL testinin Tip I hatası =  0.053
30  orneklem buyuklugu icin AD testinin Tip I hatası =  0.059
30  orneklem buyuklugu icin DAP testinin Tip I hatası =  0.062
30  orneklem buyuklugu icin JB